# Salvando dados em Big Data ETL com Apache Spark

Após executar as tarefas que compõem a fase de Transformação, os dados estão prontos para serem carregados para repositório de destino.
Por isto, esta etapa consiste em fisicamente estruturar e carregar os dados para dentro do repositório de destino. Dependendo das necessidades da empresa, este processo varia amplamente. Em alguns casos, as novas informações podem substituir as existentes semanalmente, com dados cumulativos e atualizados, ao passo que outros sistemas podem adicionar dados a cada hora. A latência e o alcance de reposição ou acréscimo constituem opções de projeto estratégicas que dependem do tempo disponível e das necessidades de negócio.
É importante entender que a frequência pode ter um impacto negativo no desempenho do sistema de dados de destino. Este pode ter dificuldade em executar as suas tarefas, já que o próprio processamento pode ficar mais lento à medida que os dados vão sendo armazenados. Assim sendo, é necessário garantir que o carregamento seja efetuado de forma a causar o menor impacto possível no sistema de dados de destino.

Existe uma variante do processo de ETL, que se designa de ELT (Extract, Load and Transform), que desempenha as mesmas funções de um processo ETL, mas em ordem diferente. 
Num processo ELT ao invés de transformar os dados antes de estes serem armazenados no repositorio de destino, o que se faz é aproveitar o sistema de destino para fazer a transformação. Esta variante faz sentido quando o repositório de destino é algo como um HDFS que tem capacidade de suportar a transformação da informação. Utilizar processos de ELT em Big Data traz benefícios em termos de desempenho e facilidade de escalabilidade. Além disso, o processo de ELT é mais flexível que o ETL, pois permite adicionar novas transformações de forma mais fácil sobre os dados brutos. Muitas das ferramentas de ETL tradicionais também oferecem esta vertente de ELT, permitindo assim ao utilizador utilizar essas duas variantes dependendo de qual for melhor para determinada situação.

Ferramentas ETL padrões talvez sejam as melhores opções nas seguintes situações:
- Quando você desejar fazer uma extensa limpeza nos dados antes de carregar no sistema de destino. ETL é a melhor solução porque você não deseja mover dados indesejáveis para o destino.
- Quando você estiver trabalhando com dados estruturados, já que as ferramentas ETL tradicionais são muito eficientes nestes casos.
- Quando você deseja enriquecer seus dados movendo para o sistema de destino, por exemplo, com uma informação de geolocalização.

## Salvando no formato CSV

Esta Seção apresenta como salvar os dados no formato CSV utilizando o Apache Spark. As principais opções na escrita dos arquivos são listadas abaixo:

- **path**: caminho do arquivo de saída.
- **header**: valor *true* para escrever o cabeçalho na primeira linha do arquivo (default: *false*).
- **delimiter**: caractere delimitador das colunas (default: *,*). 
- **quote**: pode ser configurado com qualquer caractere para delimitar um texto. Delimitadores dentro das *quotes* serão ignoradas (default: *"*).
- **escape**: pode ser configurado como qualquer caractere para escape no texto. Caracteres *quote* serão ignorados (default: *\*).
- **nullValue**: string para indicar um valor null. Os valores null no DataFrame serão escritos como string.
- **dateFormat**: string que indica o formato da data seguindo os padrões definidos pelo [java.text.SimpleDateFormat](https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html). Se não for específicado, é utilizado o formato yyyy-MM-dd HH:mm:ss.S.
- **compression**: formato de compressão na escrita. Pode ser utilizado o nome curto minúsculo dos formatos de compressão (bzip2, gzip, lz4 e snappy). (default: sem compressão).
- **quoteMode**: quando colocar *quotes* nos campos (ALL, MINIMAL (padrão), NON_NUMERIC, NONE). Você pode ver mais detalhes em [Quote Modes](https://commons.apache.org/proper/commons-csv/apidocs/org/apache/commons/csv/QuoteMode.html).
- **mode**: define como lidar com os dados se já existirem no sistema de destino. Mais detalhes podem ser visto na [documentação do Spark](https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html#save-modes). [default: 'errorifexists']

In [3]:
#cria um DataFrame com duas colunas: linguagem e porcentagem de adoção de cada linguagem
df = spark.createDataFrame([("Scala", 15), ("SQL", 15), ("Python", 25), ("R", 5), ("Java", 40)], ["linguagem", "adocao_perc"])
display(df)

linguagem,adocao_perc
Scala,15
SQL,15
Python,25
R,5
Java,40


In [4]:
# escreve o arquivo csv com as opções padrões. Escrita com overwrite para substituir caso já exista o arquivo.
df.write.csv('file:///tmp/linguagem.csv', mode='overwrite')

# o arquivo não possui cabeçalho
display(spark.read.csv('file:///tmp/linguagem.csv'))

_c0,_c1
Python,25
Scala,15
Java,40
SQL,15
R,5


In [5]:
# escreve o arquivo csv com as opções padrões. Escrita com overwrite para substituir caso já exista o arquivo.
df.write.csv('file:///tmp/linguagem.csv', mode='overwrite', header=True)

# o arquivo agora possui cabeçalho
display(spark.read.csv('file:///tmp/linguagem.csv', header=True))

linguagem,adocao_perc
Python,25
Scala,15
Java,40
SQL,15
R,5


In [6]:
# escreve o arquivo com compressão bzip2
df.write.csv('file:///tmp/linguagem_codec.csv', compression='bzip2')

In [7]:
%sh
ls -l /tmp/linguagem_codec.csv

total 28
-rw-r--r-- 1 root root 576 Apr 26 12:51 _committed_2738912858925636977
-rw-r--r-- 1 root root 14 Apr 26 12:51 part-00000-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2315-c000.csv.bz2
-rw-r--r-- 1 root root 51 Apr 26 12:51 part-00001-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2316-c000.csv.bz2
-rw-r--r-- 1 root root 49 Apr 26 12:51 part-00003-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2318-c000.csv.bz2
-rw-r--r-- 1 root root 54 Apr 26 12:51 part-00004-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2319-c000.csv.bz2
-rw-r--r-- 1 root root 46 Apr 26 12:51 part-00006-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2321-c000.csv.bz2
-rw-r--r-- 1 root root 52 Apr 26 12:51 part-00007-tid-2738912858925636977-c0e4007b-4690-4168-bf09-274f7c8a865d-2322-c000.csv.bz2
-rw-r--r-- 1 root root 0 Apr 26 12:51 _started_2738912858925636977
-rw-r--r-- 1 root root 0 Apr 26 12:51 _SUCCESS

## Salvando dados no formato JSON

Esta Seção apresenta alguns exemplos de escrita dos dados no formato JSON. Os dados são visualizados, posteriormente, com o comando *head* do linux.

In [10]:
#escrevendo o DataFrame no formato JSON
df.write.json("file:///tmp/linguagem.json", mode='overwrite')

In [11]:
%sh
head /tmp/linguagem.json/*.json

==> /tmp/linguagem.json/part-00000-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1908-c000.json <==

==> /tmp/linguagem.json/part-00001-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1909-c000.json <==
{"linguagem":"Scala","adocao_perc":15}

==> /tmp/linguagem.json/part-00003-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1911-c000.json <==
{"linguagem":"SQL","adocao_perc":15}

==> /tmp/linguagem.json/part-00004-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1912-c000.json <==
{"linguagem":"Python","adocao_perc":25}

==> /tmp/linguagem.json/part-00006-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1914-c000.json <==
{"linguagem":"R","adocao_perc":5}

==> /tmp/linguagem.json/part-00007-tid-2755608107799557609-d1c7d25e-6328-40ab-b477-643cc065c6c6-1915-c000.json <==
{"linguagem":"Java","adocao_perc":40}

## Salvando os dados no formato Parquet

Esta Seção apresenta algumas opções de escrita em arquivos no formato tabular com o Spark.

In [13]:
# escrevendo o arquivo no formato Parquet
df.write.parquet("file:///tmp/linguagem.parquet", mode='overwrite')
display(spark.read.parquet("file:///tmp/linguagem.parquet"))

linguagem,adocao_perc
Python,25
Scala,15
Java,40
SQL,15
R,5


In [14]:
%sh
ls -l /tmp/linguagem.parquet

total 28
-rw-r--r-- 1 root root 618 Apr 28 13:12 _committed_5093076830301948567
-rw-r--r-- 1 root root 381 Apr 28 13:12 part-00000-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1916-c000.snappy.parquet
-rw-r--r-- 1 root root 702 Apr 28 13:12 part-00001-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1917-c000.snappy.parquet
-rw-r--r-- 1 root root 680 Apr 28 13:12 part-00003-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1919-c000.snappy.parquet
-rw-r--r-- 1 root root 711 Apr 28 13:12 part-00004-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1920-c000.snappy.parquet
-rw-r--r-- 1 root root 662 Apr 28 13:12 part-00006-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1922-c000.snappy.parquet
-rw-r--r-- 1 root root 691 Apr 28 13:12 part-00007-tid-5093076830301948567-56760230-0585-4925-ad68-8a289c5c7d30-1923-c000.snappy.parquet
-rw-r--r-- 1 root root 0 Apr 28 13:12 _started_5093076830301948567
-rw-r--r-- 1 root root 0 Apr 28 13:12 _SUCCESS

É possível escrever a saída particionando a saída por alguma coluna do DataFrame, como no exemplo abaixo:

In [16]:
# salvando o DataFrame no formato Parquet particionando pela coluna 'linguagem'
df.write.partitionBy("linguagem").parquet("file:///tmp/linguagem_partition.parquet", mode='overwrite')

Veja que foram criados cinco diretórios (número de linguagens distintas), cada um contendo os dados da partição referente a linguagem:

In [18]:
%sh
ls /tmp/linguagem_partition.parquet/linguagem=Java

_committed_2908462342022193907
part-00007-tid-2908462342022193907-4a8f8421-85ac-4afc-8ca4-bd8a537fc81c-1938.c000.snappy.parquet
_started_2908462342022193907
_SUCCESS

## Considerações Finais

Esta Seção apresentou como salvar os dados no formato CSV e Parquet. O Spark possui vários outros sistemas destino suportados que podem ser vistos com mais detalhes da documentação da [Databricks](https://docs.databricks.com/spark/latest/data-sources/index.html) e do [Apache Spark](https://spark.apache.org/docs/latest/sql-data-sources.html).

Você poderá encontrar vários outros exemplos de ETL em Big Data com outras ferramentas, como Apache Sqoop, Apache Flume, Apache Kafka no meu [repositório do github](https://github.com/savioteles/big_data). Neste repositório você vai encontrar os códigos e um tutorial de como executar cada estudo de caso.